In [1]:
from googleapiclient.discovery import build
import re
import os
from dotenv import load_dotenv
import csv

In [3]:
load_dotenv()
API_KEY = os.getenv("API_Key")

# ====== 建立 YouTube API 客戶端 ======
youtube = build('youtube', 'v3', developerKey=API_KEY)

def load_lines_from_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        return [line.strip() for line in f if line.strip()]
def get_all_comments(video_id):
    comments = []
    next_page_token = None

    while True:
        response = youtube.commentThreads().list(
            part='snippet,replies',
            videoId=video_id,
            maxResults=100,
            pageToken=next_page_token,
            textFormat='plainText'
        ).execute()

        for item in response['items']:
            top_comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
            comments.append(top_comment)

            if 'replies' in item:
                for reply in item['replies']['comments']:
                    reply_text = reply['snippet']['textDisplay']
                    comments.append(reply_text)

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break

    return comments

# ========== 工具：統計關鍵字 ==========
def count_keywords(comments, keywords):
    keyword_counts = {kw: 0 for kw in keywords}
    for comment in comments:
        for kw in keywords:
            if re.search(re.escape(kw), comment):
                keyword_counts[kw] += 1
                #print(f'找到關鍵字「{kw}」於留言：\n{comment}\n')
    return keyword_counts

# ========== 主程式 ==========
if __name__ == '__main__':
    STUDENT_FILE = os.getenv("STUDENT_FILE")
    VIDEO_ID_FILE = os.getenv("VIDEO_ID_FILE")
    keywords = load_lines_from_file(STUDENT_FILE)
    video_ids = load_lines_from_file(VIDEO_ID_FILE)

    total_counts = {kw: 0 for kw in keywords}
    results = []  # 儲存每支影片的統計

    for vid in video_ids:
        print(f"\n處理影片 ID：{vid}")
        try:
            comments = get_all_comments(vid)
            print(f"共抓取 {len(comments)} 筆留言")
            video_counts = count_keywords(comments, keywords)

            # 累加總計
            for kw in keywords:
                total_counts[kw] += video_counts[kw]

            # 儲存當前影片的結果
            row = {'影片ID': vid}
            row.update(video_counts)
            #results.append(row)

        except Exception as e:
            print(f"影片 {vid} 發生錯誤：{e}")
            # 加入錯誤影片到結果中
            row = {'影片ID': vid}
            row.update({kw: '錯誤' for kw in keywords})
            #需要加入錯誤
            #results.append(row)

    # 最後加入總計列
    total_row = {'影片ID': '總計'}
    total_row.update(total_counts)
    results.append(total_row)

    # ========== 輸出 CSV ==========
    output_file = 'result.csv'
    with open(output_file, 'w', newline='', encoding='utf-8-sig') as csvfile:
        fieldnames = ['影片ID'] + keywords
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(results)

    print(f"\n所有結果已輸出至 {output_file}")


處理影片 ID：20EEMxKKD0s
共抓取 26 筆留言

處理影片 ID：VyQuufedhH8
共抓取 17 筆留言

處理影片 ID：m7WkSOR7Cq4
共抓取 23 筆留言

處理影片 ID：M9ZH6aVXVII
共抓取 25 筆留言

處理影片 ID：52kTMWIcStM
共抓取 24 筆留言

處理影片 ID：vkKSGu4_e-4
共抓取 22 筆留言

處理影片 ID：v94G0bUrNU4
共抓取 33 筆留言

處理影片 ID：LvukZKmYtrE
共抓取 23 筆留言

處理影片 ID：K8hHhtMEGfE
共抓取 25 筆留言

處理影片 ID：biySJxUVu1o
共抓取 19 筆留言

處理影片 ID：P50Da2Falo4
共抓取 23 筆留言

處理影片 ID：JJ5vI5YMOdg
共抓取 21 筆留言

處理影片 ID：VzXyeKYdlBY
共抓取 22 筆留言

處理影片 ID：scOPrDrdFrk
共抓取 23 筆留言

處理影片 ID：O0V_hBrs2DY
共抓取 15 筆留言

處理影片 ID：DZdeAfwQ1fw
共抓取 23 筆留言

處理影片 ID：wEy6uv399Aw
共抓取 24 筆留言

所有結果已輸出至 result.csv
